In [1]:
from Preprocessor import Preprocessor

#FilterReview
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import GaussianNB 
from sklearn import metrics 
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import pickle 
import os.path
'''
readCSV :CSV 파일을 읽는다. 고정포맷이 정해짐 
trainModel(ts =0.25): 입력된 모델에서 학습시킨다. ts 테스트 케이스 
predict(): 학습시킨 모델에 test데이터가 입력되면 예측한다. 
writePredictionResult(): 예측 결과를 파일로 출력한다. 
'''
class FilterReviewSV:
    def __init__(self):
        
        self.model_filename = 'model/FilterReviewSV_model'
        self.isSetModel = False
        if not(self.isSetModel) and os.path.isfile(self.model_filename+'.sav'):
            self.gnb = pickle.load(open(self.model_filename+'.sav', 'rb'))
            
    def input_X(self, para):
        pre = Preprocessor()
        pre.setPara(para)
        self.X = pre.transVector()
    def input_Y(self, result):
        self.Y = result
        
    def readCSV(self,filename,tag=True):
        self.data = pd.read_csv(filename,encoding='euc-kr')
        self.X_content = self.data['글'].values
        self.input_X(self.X_content)
        if tag == True:
            self.input_Y(self.data['tag'].values)
        
    def trainModel(self,ts):
        self.X_train,self.X_test,self.y_train,self.y_test=train_test_split(self.X,self.Y,test_size=ts)
        self.gnb = GaussianNB()
        self.y_pred = self.gnb.fit(self.X_train, self.y_train).predict(self.X_test)

        # 테스트 socre 확인
        print("Number of right labeled points out of a total %d points : %.2f %%"
      % (len(self.X),100*(len(self.X)-(self.y_test != self.y_pred).sum())/len(self.X)))
        
        pickle.dump(self.gnb, open(self.model_filename+'.sav', 'wb'))
        isSetModel = True
    def predict(self):
        self.y_pred = self.gnb.predict(self.X)
        self.df = pd.DataFrame({'p_tag':self.y_pred, '글':self.X_content})
        return self.y_pred
    def writePredictionResult(self, filename):
        self.df.to_csv(filename,encoding='euc-kr')
        


C:\Users\JH\Anaconda3\envs\farm\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\JH\Anaconda3\envs\farm\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
fr = FilterReviewSV()
#fr.input_X('나는 멍청이다.\n취업 좀 하자\n취업 좀 하자\n취업 좀 하자')
#fr.input_Y([1,0,1,0])
#fr.readCSV(tag=True,filename='Train.csv')
#fr.trainModel(ts=0.25)

fr.readCSV(tag=False,filename='Test.csv')
fr.predict()

fr.writePredictionResult('sample.csv')
print(fr.y_pred)

['나는 멍청하다.' '지수는 왜 나갔을까?' '언제 취업하지…' '잠온다.']
[1 0 1 1]
